## Calculate Phi vertical mean for a particular layer and LME

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]

In [3]:
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
filename = f'{datadir}/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.TEMP.024901-031812.nc'
dset = xr.open_dataset(filename)
dz = dset['dz'].isel(z_t=range(0, 35))/1e2 # m
darea = dset['TAREA'].isel(nlat=range(a1, a2), nlon=range(b1, b2))/1e10 #km*km
KMT = dset['KMT'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
dset.close()

In [4]:
dvol = dz*darea
for xx in range(a2-a1):
    for yy in range(b2-b1):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            dvol[kk:, xx, yy] = float("nan")

In [5]:
filename = f'{path}/DPLE_driftcorrected_Phi_mean_dimension_reduced_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in range(13):
    Phi_i = ds[f'Phi_{ii}']*dvol
    # select a vertical layer
    dsf[f'Phi_{ii}_0-200m'] = Phi_i.isel(z_t=range(0, 20, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 20, 1)).sum(dim='z_t'))
    dsf[f'Phi_{ii}_200-600m'] = Phi_i.isel(z_t=range(20, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(20, 35, 1)).sum(dim='z_t'))

In [6]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:          (nlat: 140, nlon: 195, L: 122)
Coordinates:
    ULONG            (nlat, nlon) float64 101.7 102.9 104.0 ... 319.5 320.0
    ULAT             (nlat, nlon) float64 15.13 15.13 15.14 ... 70.63 70.63
    Y                int64 1954
  * L                (L) int64 1 2 3 4 5 6 7 8 ... 116 117 118 119 120 121 122
    TLAT             (nlat, nlon) float64 14.96 14.96 14.97 ... 70.42 70.42
    TLONG            (nlat, nlon) float64 101.2 102.3 103.4 ... 319.2 319.7
Dimensions without coordinates: nlat, nlon
Data variables: (12/26)
    Phi_0_0-200m     (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_0_200-600m   (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_1_0-200m     (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_1_200-600m   (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_2_0-200m     (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_2_200-600m   (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    ...               ...
    Phi_10_0-200m    (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_10_200-600m  (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_11_0-200m    (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_11_200-600m  (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_12_0-200m    (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_12_200-600m  (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [7]:
%time
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_mean_dimension_reduced_vertical_mean_ens_mean_monthly.nc', mode='w')

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


## Calculate Yearly

In [13]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """

    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

In [14]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE_driftcorrected_Phi_mean_dimension_reduced_vertical_mean_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in range(13):
    # select a vertical layer
    for layer in ['0-200m','200-600m']:
        dsf[f'Phi_{ii}_{layer}'] = xr_reshape(ds[f'Phi_{ii}_{layer}'][2:,:,:], 'L', ['year', 'mon'], [np.arange(10), np.arange(12)]).mean(dim='mon')

In [15]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:          (nlat: 140, nlon: 195, year: 10)
Coordinates:
    ULONG            (nlat, nlon) float64 101.7 102.9 104.0 ... 319.5 320.0
    ULAT             (nlat, nlon) float64 15.13 15.13 15.14 ... 70.63 70.63
    Y                int64 1954
    TLAT             (nlat, nlon) float64 14.96 14.96 14.97 ... 70.42 70.42
    TLONG            (nlat, nlon) float64 101.2 102.3 103.4 ... 319.2 319.7
  * year             (year) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: nlat, nlon
Data variables: (12/26)
    Phi_0_0-200m     (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_0_200-600m   (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_0-200m     (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_200-600m   (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_0-200m     (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_200-600m   (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    ...               ...
    Phi_10_0-200m    (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_10_200-600m  (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_11_0-200m    (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_11_200-600m  (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_12_0-200m    (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_12_200-600m  (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan

In [16]:
%%time
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_mean_dimension_reduced_vertical_mean_ens_mean_yearly.nc', mode='w')

CPU times: user 72.8 ms, sys: 12.3 ms, total: 85.1 ms
Wall time: 427 ms
